In [1]:
%load_ext autoreload

In [71]:
%autoreload
import sys
sys.path.insert(0, '../')

In [72]:
from src.utilities.mluar_utils import *

In [73]:
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset, Dataset, load_from_disk
import numpy as np
from einops import rearrange, reduce, repeat
import torch
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib import pyplot as plt
import math
import pandas as pd
import pickle as pkl

pd.set_option('display.max_colwidth', None)


In [67]:
MULTI_LUAR_PATH =  "/mnt/swordfish-pool2/milad/multi-luar-reddit-model/"
LUAR_PATH =  "/mnt/swordfish-pool2/nikhil/LUAR/pretrained_weights/LUAR-MUD/"

In [6]:
# Load models
multiluar_model = AutoModel.from_pretrained(MULTI_LUAR_PATH, trust_remote_code=True)
luar_model = AutoModel.from_pretrained(LUAR_PATH, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("rrivera1849/LUAR-MUD")

In [7]:
# Load data
data_path = '/mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_perGenre-HRS2.1/TA2/hrs_06-27-24_english_perGenre-HRS2.1/data/hrs_06-27-24_english_perGenre-HRS2.1_TA2_input'
ground_truth_path = '/mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_perGenre-HRS2.1/TA2/hrs_06-27-24_english_perGenre-HRS2.1/groundtruth/hrs_06-27-24_english_perGenre-HRS2.1_TA2'
hiatus_data, _, _ = load_aa_data(data_path, ground_truth_path)

Loading:  /mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_perGenre-HRS2.1/TA2/hrs_06-27-24_english_perGenre-HRS2.1/data/hrs_06-27-24_english_perGenre-HRS2.1_TA2_input


In [8]:
# keep authors with only more than one text
authors_with_multiple_texts = [x[0] for x in hiatus_data.authorID.value_counts().to_dict().items() if x[1] > 1]
hiatus_data = hiatus_data[hiatus_data.authorID.isin(authors_with_multiple_texts)]

In [9]:
hiatus_data = hiatus_data.sample(2000)

In [10]:
hiatus_data_texts = hiatus_data.fullText.tolist()
labels = hiatus_data['authorID'].tolist()

# Compute embeddings
max_seq_length = 736
hiatus_data_embeddings = get_luar_embeddings(hiatus_data_texts, multiluar_model, tokenizer, max_length=max_seq_length, batch_size=1, is_multi_luar=True)

In [11]:
#np.max([len(x.split()) for x in hiatus_data.fullText.tolist()])

### Experiment Design

- For each layer, we find pairs of ground-truth texts written by the same author where the corresponding layer's embedding scored them high compared to other layers
- For each layer, we take a sample of these pairs of texts and prompt ChatGPT to find which linguistic level they are similar

#### Step 1:

In [74]:
# Compute Multi-luar similarity matrices at every layer and average of the layers
muti_luar_layers_sims = [compute_similarities(hiatus_data_embeddings, hiatus_data_embeddings, layer=i) for i in range(7)]
#muti_luar_layers_sims.append(compute_similarities(hiatus_data_embeddings, hiatus_data_embeddings, layer=None))
muti_luar_layers_sims = np.stack(muti_luar_layers_sims)

In [78]:
# Compute significant pairs to layers
layer_to_sig_pairs = {layer: extract_sig_pairs_for_layer(hiatus_data_texts, muti_luar_layers_sims, labels, layer) for layer in range(7)}

In [79]:
len(labels), len(set(labels))

(2000, 963)

In [80]:
layer_x_pairs = []
for layer in [0, 1, 2, 3, 4, 5, 6]:
    print(layer, len(layer_to_sig_pairs[layer]))
    sample_of_pairs = layer_to_sig_pairs[layer][:10]
    layer_x_pairs += [{'text-1': x[0], 'text-2': x[1], 'z-score': x[2], 'layer-sim': x[3], 'layer': layer} for x in sample_of_pairs]
layer_x_pairs_df = pd.DataFrame(layer_x_pairs)

0 0
1 0
2 0
3 0
4 0
5 0
6 0


In [63]:
layer_x_pairs_df[layer_x_pairs_df.layer == 6].head()

,text-1,text-2,z-score,layer-sim,layer
13,"Tables that leave\n\nAs a host-nothing is more awkward/uncomfortable than when a table gets up and leaves because of the service.\n\nTonight was overall a terrible shift. We had two newer servers on and one experienced server who wanted to get cut first. With newer servers it’s hard because we can’t tell how much they can handle at once until we get used to working with them.\n\nAnyways I seat a four top with a reservation and they were super nice. It was pretty busy, but Saturday nights are always busy (I work in a big city too)\n\nThey got drinks but then out of the blue they all get up to leave and tell my co-host and I they’re leaving because the server has been gone for forever and it’s not worth it. It’s such an awkward position to be in because wtf am I supposed to do other than say I’m so sorry. Plea with them to stay? Grab my manager? I actually couldn’t get my manager because she was running around like a madwoman all night.\n\nSo they left and it was just like ….\n\nThen an hour later we sat another four top with a reservation and legit same thing. They leave without even being watered (I’m not sure if they were greeted because sometimes customers flip the glasses over themselves)\n\nI felt bad for the server but my hands were tied so I had to seat the them. Sometimes I’ll check in with servers when I’ve noticed a table hasn’t been watered but sometimes they get annoyed and go “I know…” or “they can wait” But what am I supposed to say when people tell us they’re leaving because of slow service????","Does the kitchen hate the hosts?\n\nI work in a big city but our kitchen is fairly small compared to the size of the restaurant. Anyways my manager’s whole ordeal is never say no. Business is business. If an eight top comes in without a reservation it doesn’t matter you seat them like that kind of manager. One time a host turned away a 14 top, 14 top emailed manager and host got fired (for other reasons too but that was a big part of it)\n\nAnyways today at lunch we got maybe 300000 walk-ins and it was stressful but I made it work. My manager comes running out saying how the kitchen is slammed and I need to stop taking people. I’m like ok cool that’s actually good because there’s only two servers on anyways and they’re weeded.\n\nIt finally dies down and I ask my mgr if the kitchen hates me. She just goes\n\n“Um…..I mean they were dying in there they had so many tickets”\n\nSo does the kitchen hate me? And hosts in general?","[0.20142695, -0.33192047, -1.5438107, 0.014910437, -0.56804776, 0.20199834, 2.0253887]","[0.9343342, 0.9308291, 0.9228647, 0.93310845, 0.9292773, 0.934338, 0.9463211]",6
14,"The Revenant\n\nHoly actual shit.\n\n<PERSON>! <PERSON> Man!! If he doesn't win the Oscar this time, he should just honestly quit acting.\n\n<PERSON> is f***ing boss once again in an antagonist role.\n\n<PERSON> strikes again with his mind-blowing camerawork.\n\nI think I almost fell off my couch three times because of how damn suspenseful this was.\n\nThe action sequences were intense as hell.\n\nThe dialogue is more than fantastic.\n\nThe plot is very well executed.\n\nWow. Just wow.\n\nI had to contain myself while writing this review because just thinking about this film makes me absolutely hyped.\n\nThe Revenant is overwhelmingly breath-taking.","The <PERSON>\n\nGoing into this movie I had 3 worries: it was going to be extremely overrated, it wasn't going to be entertaining, and I wasn't going to understand the plot of the film. None of these things turned out to be true, and I'm very delighted by this.\n\nA film being called the best movie of all-time definitely will be called overrated by most, but this film is just excellent in every aspect. Its story is as masterful and interesting as any other great film from this day and age. The acting is beyond good, as <PERSON>, <PERSON>, and <PERSON> absolutely blew me away with their performances. Also, the action sequences and suspense 

In [54]:
layer_x_pairs_df.to_pickle('../data/layer_to_pairs_signficance.pkl')

#### Step 2:

In [55]:
from datadreamer import DataDreamer
from datadreamer.llms import HFTransformers, ParallelLLM, OpenAI
from datadreamer.steps import DataFromPrompt, ProcessWithPrompt,  HFHubDataSource, DataSource, zipped, concat
from functools import partial
from transformers import QuantoConfig
from datasets import concatenate_datasets, load_dataset
import json


model = OpenAI(model_name="gpt-4", api_key='sk-proj-zTbZNk16Ik1pZnqLn38ZT3BlbkFJImq3pd7widkr7RzsC771') #kathy's lab
# model = HFTransformers(
#                 "meta-llama/Meta-Llama-3-8B-Instruct",
#                 quantization_config=QuantoConfig(weights="int8"),
#                 device=0,
#                 device_map="cuda",
#             )
# model.config.pad_token_id = model.config.eos_token_id

def gen_from_iterable_dataset(iterable_ds):
    yield from iterable_ds

def evaluate_text_similarities(data_path, document_pairs, linguistic_lvl_name, linguistic_lvl_desc):
    instruction = "Given the two Documents below, rate their <linguistic_lvl> on a scale from 1 to 5: Score 1 equals very low similarity and score 5 equals high similarity. First, give reasons for your score and then output the score. The output should be in the following format: {\"reasons\": \"explain your rating\",  \"score\": \"<json integer>\"}"
    instruction = instruction.replace("<linguistic_lvl>", linguistic_lvl_desc)
    
    with DataDreamer(data_path):
        datasource = DataSource('documents', Dataset.from_list(document_pairs))
        datasource = datasource.map(lambda row: {'inputs': 'Document 1:\n {} \n Document 2:\n {}'.format(row['text-1'], row['text-2'])}, auto_progress=False)
        ds_focus_questions = ProcessWithPrompt(
          "{} describe text similarity".format(linguistic_lvl_name),
          inputs={"inputs": datasource.output["inputs"]},
          args={
             "llm": model,
             "n": 1,
             "instruction": instruction
          },
          outputs={"generations": linguistic_lvl_name},
        ).select_columns([linguistic_lvl_name])
        
        zipped_step = zipped(datasource, ds_focus_questions)

        results_iter = zipped_step.output.dataset
        results_ds   = Dataset.from_generator(partial(gen_from_iterable_dataset, results_iter))

        return results_ds

ModuleNotFoundError: No module named 'huggingface_hub.utils._errors'

In [8]:
layer_to_sig_pairs = pkl.load(open('../data/layer_to_pairs_signficance.pkl', 'rb'))

In [9]:
layer_x_pairs = []
for layer in [0, 1, 2, 3, 4, 5, 6]:
    print(layer, len(layer_to_sig_pairs[layer]))
    sample_of_pairs = layer_to_sig_pairs[layer][:10]
    layer_x_pairs += [{'text-1': x[0], 'text-2': x[1], 'z-score': x[2], 'layer': layer} for x in sample_of_pairs]

0 2154
1 6
2 20
3 108
4 16
5 56
6 890


In [10]:
len(layer_x_pairs)

66

In [11]:
# Given the two Documents below, rate their <linguistic_lvl> on a scale from 1 to 5: Score 1 equals very low similarity and score 5 equals high similarity. First, give reasons for your score and then output the score in the following output format: {\"reasons\": \"explain your rating\",  \"score\": \"<json integer>\"}
ling_phenomena = {
    'syntax': 'syntactic similarity',
    'semantic': 'semantic similarity',
    'lexical': 'lexical similarity',
    'discourse': 'discourse similarity'
}

In [12]:
results = []
for key, val in ling_phenomena.items():
    results.append(evaluate_text_similarities('./output', layer_x_pairs, key, val))

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: ./output
[ 🤖 DataDreamer 💤 ] Step 'documents' results loaded from disk. 🙌 It was previously run and saved.
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity' was previously run and saved, but was outdated. 😞
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity' progress: 50 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity' finished and is saved to disk. 🎉
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity (select_columns)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity (select_columns)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'zipped(documents (map), syntax describe text similarity (select_columns))' 

In [13]:
all_results = concatenate_datasets(results)

In [14]:
all_results.save_to_disk('../data/described_similarities_ds')

Saving the dataset (0/1 shards):   0%|          | 0/264 [00:00<?, ? examples/s]

### Analyze layer to lingustic similarity:

In [15]:
all_results = load_from_disk('../data/described_similarities_ds')

In [16]:
all_results_df = all_results.to_pandas()

In [17]:
zscores= all_results_df['z-score'].tolist()

In [26]:
all_results_df[['text-1', 'text-2', 'layer', 'z-score']].head(n=50)

,text-1,text-2,layer,z-score
0,"Anatomy of a Fall\n\ni'm fully aware it’s not a fair comparison, but i’d like to be able to have a foundation of a dead-body-case type work-of-fiction to base my conclusion on how weak or strong the one i’m watching currently is. recently, and in this case, i’ve come to referring to disco elysium, of which strongly elicited throughout my entire lengthy time with it what the evident culmination of anatomy of a fall - the kitchen fight recording, some of the child actors scenes - coaxed. immediately and on a purely cosmetic and surface level, Fall lacks any type of relatively incisive and personally disruptive style. this sort of flatly lit, boringly pictured and blocked, framed with little to no intention movie reminds me of Tar, from a few years ago, and at least for a little while we were happy with this natural Lightroom preset of a color palate. the question remains that if the director moved focus away from the visuals to focus on the performances, and the performances aren’t at all times captivating squarely enough to fill the vacuum, then why are you wasting our time\n\naddendum: <PERSON> states on an interview i read on cinematography world that the original intention was to shoot on 35, and they rather proceeded to shoot hawk v lite anamorphic on alexa then matte out the beautiful academy ratio to widescreen. horrific world we live in","<PERSON>\n\ngoing back and watching these in the original language instead of the goofy english dubs. a lot of people say this is the best one but i'm not really sure. most of them are pretty great. all of them are intensely watchable. this one in particular has the most complete plot with characters who are fearless and show each other mercy. on atmosphere alone the winner could go to maybe four or five of ghibli's other works. i remember watching this one as a kid pretty intensely since it's so violent and the sound effects try to reduce the gore to these cartoonish kinda tom and jerry actions but theres still so much blood. even when i was 10 or 12 i begin to notice that there weren't really characters like <PERSON>'s in american children's media, like role models that had personal resolve but that were also deeply flawed. cartoons here are much more concerned with being sardonic or melodramatic. the part where san baby birds the jerky in the main dude's mouth always gets me",0,"[1.6476448774337769, -1.2187529802322388, -0.9693524241447449, 0.131813183426857, -0.2457742691040039, -0.5583204627037048, 1.21274995803833]"
1,"The Power of the Dog\n\nin isolation there are very strong parts, unfortunately connected together with poorly directed ones that kind of meander without direction. the best scenes are cut too short and show nothing and the badly filmed ones always stretch way too long and show way too much. this film as a whole explores who or what killed <PERSON>. we know, because we've seen every part of the chain that leads to his death. however, where the entire thing fails is that it almost seems like we know more than the movie does","<PERSON>\n\ngoing back and watching these in the original language instead of the goofy english dubs. a lot of people say this is the best one but i'm not really sure. most of them are pretty great. all of them are intensely watchable. this one in particular has the most complete plot with characters who are fearless and show each other mercy. on atmosphere alone the winner could go to maybe four or five of ghibli's other works. i remember watching this one as a kid pretty intensely since it's so violent and the sound effects try to reduce the gore to these cartoonish kinda tom and jerry actions but theres still so much blood. even when i was 10 or 12 i begin to notice that there weren't really characters like <PERSON>'s in american children's media, like role models that had personal resolve but that were also deeply flawed. cartoons here are much more concerned with being sardonic or melodramatic. the part where

In [20]:
all_results_df.layer.value_counts()

layer
0    40
2    40
3    40
4    40
5    40
6    40
1    24
Name: count, dtype: int64

In [23]:
def aggregate_score(list_of_scores):
    scores = []
    for json_str in list_of_scores:
        if json_str == None:
            continue
        try:
            if "\n\n" in json_str:
                json_str = json_str.split("\n")[-1].strip()

            json_str = json_str.replace("Output:", "").strip()
                
                
            json_obj = json.loads(json_str)            
            scores.append(int(json_obj['score']))
        except:
            print(json_str)
            print('==============')
    return round(np.median(scores), 2)

# Aggregating layer score
layer_scores_df = all_results_df.groupby('layer').aggregate({
    'lexical': aggregate_score,
    'syntax': aggregate_score,
    'discourse': aggregate_score,
    'semantic': aggregate_score,
}).reset_index()

In [24]:
layer_scores_df.head(n=7)

,layer,lexical,syntax,discourse,semantic
0,0,1.0,2.0,2.0,1.0
1,1,2.0,3.0,3.0,2.0
2,2,1.0,3.0,2.0,1.0
3,3,2.0,3.0,2.0,2.0
4,4,2.0,3.0,2.0,2.0
5,5,2.0,3.0,2.5,2.0
6,6,2.0,3.0,2.5,2.0
